<a href="https://colab.research.google.com/github/sunnysavita10/Generative-AI-Indepth-Basic-to-Advance/blob/main/RAG_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Fusion

In [9]:
!pip -q install langchain huggingftiktace_hub oken pypdf
!pip -q install google-generativeai chromadb
!pip -q install sentence_transformers

ERROR: Could not find a version that satisfies the requirement huggingftiktace_hub (from versions: none)
ERROR: No matching distribution found for huggingftiktace_hub


In [10]:
!pip -q install -U langchain-community

### Download the Data & Utils

In [11]:
import textwrap
def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


In [12]:
import os
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [13]:
%pip install --upgrade --quiet  langchain-google-genai

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [15]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

(Verse 1)
In a world of data, vast and deep,
Where knowledge flows, and secrets sleep,
A tool arose, a shining light,
LangChain its name, a beacon bright.

(Verse 2)
With chains of prompts, it weaves a spell,
Unlocking wisdom, stories to tell,
From ancient texts to modern lore,
It gathers knowledge, evermore.

(Verse 3)
Models grand, it calls its own,
Bard and GPT, on their throne,
With whispers soft, it guides their hand,
To craft responses, wise and grand.

(Verse 4)
For developers, a treasure found,
To build applications, safe and sound,
With agents swift, and tools refined,
LangChain empowers, every mind.

(Verse 5)
No longer lost, in data's maze,
LangChain guides us, through the haze,
With clarity and purpose strong,
It leads us forward, all along.

(Verse 6)
So let us sing, of LangChain's might,
A beacon shining, in the digital night,
May its chains of knowledge, forever bind,
The power of language, for all mankind. 



## Google

## Imports

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
import langchain

## Load in Docs

In [17]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')

In [20]:
data_path="/content/data"

In [21]:
# !pip -q install unstructured
!pip -q install "unstructured[pdf]"

In [22]:
%%time
loader = DirectoryLoader(data_path, glob="*.pdf", show_progress=True)
docs = loader.load()

100%|██████████| 2/2 [00:24<00:00, 12.05s/it]

CPU times: user 14.8 s, sys: 1.23 s, total: 16.1 s
Wall time: 24.1 s


In [23]:
len(docs)

2

In [24]:
docs = docs[:50]
len(docs)

2

In [25]:
docs[0]

Document(metadata={'source': '/content/data/Transformer.pdf'}, page_content='3 2 0 2\n\ng u A 2\n\n] L C . s c [\n\n7 v 2 6 7 3 0 . 6 0 7 1 : v i X r a\n\nProvided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.\n\nAttention Is All You Need\n\nAshish Vaswani∗ Google Brain avaswani@google.com\n\nNoam Shazeer∗ Google Brain noam@google.com\n\nNiki Parmar∗ Google Research nikip@google.com\n\nJakob Uszkoreit∗ Google Research usz@google.com\n\nLlion Jones∗ Google Research llion@google.com\n\nAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu\n\nŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com\n\nIllia Polosukhin∗ ‡ illia.polosukhin@gmail.com\n\nAbstract\n\nThe dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder th

In [27]:
print(docs[1].page_content)

1 2 0 2

r p A 2 1

] L C . s c [

4 v 1 0 4 1 1 . 5 0 0 2 : v i X r a

Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

Patrick Lewis†‡, Ethan Perez(cid:63),

Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,

Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†

†Facebook AI Research; ‡University College London; (cid:63)New York University; plewis@fb.com

Abstract

Large pre-trained language models have been shown to store factual knowledge in their parameters, and achieve state-of-the-art results when ﬁne-tuned on down- stream NLP tasks. However, their ability to access and precisely manipulate knowl- edge is still limited, and hence on knowledge-intensive tasks, their performance lags behind task-speciﬁc architectures. Additionally, providing provenance for their decisions and updating their world knowledge remain open research problems. Pre- trained models with a differentiable access mechanism to e

In [ ]:
print(docs[1].page_content)

In [28]:
raw_text = ''
for i, doc in enumerate(docs):
    text = doc.page_content
    if text:
        raw_text += text

In [29]:
print(raw_text)

3 2 0 2

g u A 2

] L C . s c [

7 v 2 6 7 3 0 . 6 0 7 1 : v i X r a

Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.

Attention Is All You Need

Ashish Vaswani∗ Google Brain avaswani@google.com

Noam Shazeer∗ Google Brain noam@google.com

Niki Parmar∗ Google Research nikip@google.com

Jakob Uszkoreit∗ Google Research usz@google.com

Llion Jones∗ Google Research llion@google.com

Aidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu

Łukasz Kaiser∗ Google Brain lukaszkaiser@google.com

Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com

Abstract

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely o

In [30]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex = False,
)

In [31]:
texts = text_splitter.split_text(raw_text)

In [32]:
len(texts)

328

In [33]:
print(texts[4])

single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.


## BGE Embeddings

In [34]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [35]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

In [36]:
embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Vector DB

In [37]:
%%time
### Make the chroma and persiste to disk
db = Chroma.from_texts(texts,embedding_function,persist_directory="./chroma_db")

CPU times: user 2.58 s, sys: 201 ms, total: 2.78 s
Wall time: 3.37 s


In [38]:
query = "Tell me about Multi Head Attention"

db.similarity_search(query, k=5)

[Document(page_content='Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.\n\nMultiHead(Q, K, V ) = Concat(head1, ..., headh)W O\n\nwhere headi = Attention(QW Q\n\ni , KW K i\n\n, V W V\n\ni )\n\nWhere the projections are parameter matrices W Q and W O ∈ Rhdv×dmodel.\n\ni ∈ Rdmodel×dk , W K\n\ni ∈ Rdmodel×dk , W V\n\ni ∈ Rdmodel×dv'),
 Document(page_content='The Transformer uses multi-head attention in three different ways:\n\nIn "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9].'),
 Document(page_content='i ∈ Rdmodel×dk , W K\n\ni ∈ Rdmodel×dk ,

## Setup a Retriever

In [40]:
retriever = db.as_retriever() # can add mmr fetch_k=20, search_type="mmr"

retriever.get_relevant_documents(query)

[Document(page_content='Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.\n\nMultiHead(Q, K, V ) = Concat(head1, ..., headh)W O\n\nwhere headi = Attention(QW Q\n\ni , KW K i\n\n, V W V\n\ni )\n\nWhere the projections are parameter matrices W Q and W O ∈ Rhdv×dmodel.\n\ni ∈ Rdmodel×dk , W K\n\ni ∈ Rdmodel×dk , W V\n\ni ∈ Rdmodel×dv'),
 Document(page_content='The Transformer uses multi-head attention in three different ways:\n\nIn "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9].'),
 Document(page_content='i ∈ Rdmodel×dk , W K\n\ni ∈ Rdmodel×dk ,

## Chat chain

In [41]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

In [42]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [43]:
prompt = ChatPromptTemplate.from_template(template)

In [44]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [45]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x79ce9f8c73d0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x79ce9f716710>, default_metadata=())

In [46]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [47]:
text_reply = chain.invoke("Tell me about Multi Head Attention")

print(wrap_text(text_reply))

Multi-head attention, a mechanism within the Transformer model, enables the model to focus
on information from different aspects and positions within the input data.

Here's a breakdown:

* **Purpose:** Instead of averaging representations, which can obscure important nuances,
multi-head attention allows for attending to various representation subspaces
simultaneously.
* **Mechanism:** It utilizes multiple "heads" that operate in parallel. Each head
calculates a scaled dot-product attention, focusing on specific relationships between
queries, keys, and values within the data.
* **Output:** The outputs from all heads are concatenated and transformed into a final
representation.
* **Benefits:** This approach helps the model capture richer and more contextualized
representations compared to single-head attention.

The provided text highlights that the Transformer specifically leverages multi-head
attention in three ways: encoder-decoder attention, self-attention within the encoder, and
se

## With RagFusion

In [48]:
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

In [49]:
prompt = ChatPromptTemplate(input_variables=['original_query'],
                            messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],template='You are a helpful assistant that generates multiple search queries based on a single input query.')),
                            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])

In [51]:
original_query = "Tell me about Multi Head Attention"

In [52]:
generate_queries = (
    prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))
)

In [53]:
generate_queries

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generates multiple search queries based on a single input query.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x79ce9f8c73d0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x79ce9f716710>, default_metadata=())
| StrOutputParser()
| RunnableLambda(lambda x: x.split('\n'))

In [54]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [55]:
ragfusion_chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [56]:
langchain.debug = True

In [57]:
ragfusion_chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}},
 'required': ['question']}

In [63]:
ragfusion_chain.invoke({"question": original_query})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Tell me about Multi Head Attention"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "Tell me about Multi Head Attention"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful assistant that generates multiple search queries based on a single input query.\nHuman: Generate multiple search queries related to: Tell me about Multi Head Attention \n OUTPUT (4 queries):"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] [1.85s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here are 4 search queries related to \"Tell me about Multi Head Attention\":\n\n1. **Multi Head Attention explained**

[(Document(page_content='Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.\n\nMultiHead(Q, K, V ) = Concat(head1, ..., headh)W O\n\nwhere headi = Attention(QW Q\n\ni , KW K i\n\n, V W V\n\ni )\n\nWhere the projections are parameter matrices W Q and W O ∈ Rhdv×dmodel.\n\ni ∈ Rdmodel×dk , W K\n\ni ∈ Rdmodel×dk , W V\n\ni ∈ Rdmodel×dv'),
  0.08225806451612902),
 (Document(page_content='The Transformer uses multi-head attention in three different ways:\n\nIn "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9].'),
  0.04945355191256831),
 (Document(pag

In [59]:
from langchain.schema.runnable import RunnablePassthrough
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [60]:
full_rag_fusion_chain.input_schema.schema()

{'title': 'RunnableParallel<context,question>Input',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [61]:
full_rag_fusion_chain.invoke({"question": "Tell me about Multi Head Attention"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Tell me about Multi Head Attention"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "question": "Tell me about Multi Head Attention"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Tell me about Multi Head Attention"
}[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "question": "Tell me about Multi Head Attention"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "question": "Tell me about Multi Head Attention"
}

[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence > prompt:ChatPr

'Multi-head attention enables the model to focus on information from different representation subspaces at different positions simultaneously. This contrasts with single-head attention, where averaging would hinder this capability.\n\nEach "head" in multi-head attention can be thought of as a separate attention mechanism that focuses on a specific aspect of the input. The outputs from all heads are then combined to produce a final representation.\n\nMathematically:\n\nMultiHead(Q, K, V) = Concat(head1, ..., headh)WO\n\nwhere:\n- headi = Attention(QWQi, KWKi, VWVi)\n- Q, K, and V represent the queries, keys, and values, respectively.\n- WQi, WKi, and WVi are parameter matrices for each head.\n- WO is a parameter matrix for the final output.\n\nThe dimensions of these matrices are:\n- WQi ∈ Rdmodel×dk\n- WKi ∈ Rdmodel×dk\n- WVi ∈ Rdmodel×dv\n- WO ∈ Rhdv×dmodel\n\nThis approach allows the model to learn richer representations compared to using a single attention head. \n'

# Rag Fusion Output:
Multi-head attention enables the model to focus on information from different representation subspaces at different positions simultaneously. This contrasts with single-head attention, where averaging would hinder this capability.

Each "head" in multi-head attention can be thought of as a separate attention mechanism that focuses on a specific aspect of the input. The outputs from all heads are then combined to produce a final representation.

Mathematically:

MultiHead(Q, K, V) = Concat(head1, ..., headh)WO

where:
- headi = Attention(QWQi, KWKi, VWVi)
- Q, K, and V represent the queries, keys, and values, respectively.
- WQi, WKi, and WVi are parameter matrices for each head.
- WO is a parameter matrix for the final output.

The dimensions of these matrices are:
- WQi ∈ Rdmodel×dk
- WKi ∈ Rdmodel×dk
- WVi ∈ Rdmodel×dv
- WO ∈ Rhdv×dmodel

This approach allows the model to learn richer representations compared to using a single attention head.


# Without Rag Fusion:
Multi-head attention, a mechanism within the Transformer model, enables the model to focus
on information from different aspects and positions within the input data.

Here's a breakdown:

* **Purpose:** Instead of averaging representations, which can obscure important nuances,
multi-head attention allows for attending to various representation subspaces
simultaneously.
* **Mechanism:** It utilizes multiple "heads" that operate in parallel. Each head
calculates a scaled dot-product attention, focusing on specific relationships between
queries, keys, and values within the data.
* **Output:** The outputs from all heads are concatenated and transformed into a final
representation.
* **Benefits:** This approach helps the model capture richer and more contextualized
representations compared to single-head attention.

The provided text highlights that the Transformer specifically leverages multi-head
attention in three ways: encoder-decoder attention, self-attention within the encoder, and
self-attention within the decoder. However, the specifics of these applications are not
fully explained within the provided excerpts.